# Imports

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install scikit-learn-extra

     |████████████████████████████████| 2.0MB 8.0MB/s 
     |████████████████████████████████| 890kB 33.2MB/s 
     |████████████████████████████████| 3.2MB 52.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0a72c1f983f9522a62b3be448b2a80f0c703177485a08f1d1611be921af9d563
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 194kB 9.0MB/s 
     |████████████████████████████████| 245kB 15.0MB/s 
     |████████████████████████████████| 112kB 18.5MB/s 
     |████████████████████████████████| 1.2MB 7.8MB/s 
     |████████████████████████████████| 624kB 7.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-learn-extra: filename=scikit_learn_extra-0.1.0b2-cp37-cp37m-linux_x86_64.whl size=339934 sha256=01032c697

In [ ]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import argparse
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import os
import random
import shutil
import time
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from tqdm import tqdm
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
from PIL import Image
import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import sys
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.gaussian_process.kernels import RBF
import torch
from scipy.stats import multivariate_normal
import  scipy.stats as st
from matplotlib import cm
import torch.optim as optim
from __future__ import print_function
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW, AutoModel
import nltk
from torch.utils.data import DataLoader
import pickle
import spacy
from gensim import corpora, models, similarities
import gensim
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


# HotpotQA


In [ ]:
DISTRACTORS_PARS_LEN = 0

In [ ]:
from datasets import load_dataset

dataset = load_dataset("hotpot_qa", 'distractor')

Reusing dataset hotpot_qa (/root/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/2079f58c1c29624e56735ac2c00efe8808ce52a014e1297552384edae85e3e6d)


In [ ]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']

filtering steps:

1.    remove yes-no questions
2.   join sentences together
3. keep only 1 distractor paragraphs 




In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")

In [ ]:

def get_token_lenght(context, question):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    return len(inputs['input_ids'][0])

def remove_distractors(example):
    all_pars = example['context']['title']
    gold_pars = list(set(example['supporting_facts']['title']))
    distractor_pars = list(set(all_pars) - set(gold_pars))
    # get indices to keep from disractors
    if len(distractor_pars) == 0 or DISTRACTORS_PARS_LEN == 0:
        distract_indices = []
    else:
        distract_indices = random.sample(range(len(distractor_pars)), DISTRACTORS_PARS_LEN)
    distractor_pars = [distractor_pars[idx] for idx in distract_indices]
    keep_pars = gold_pars + distractor_pars
    keep_pars_indices = [all_pars.index(keep_par) for keep_par in keep_pars]
    example['context']['title'] = [example['context']['title'][idx] for idx in keep_pars_indices]
    example['context']['sentences'] = ["".join(example['context']['sentences'][idx]) for idx in keep_pars_indices] 

    question = example['question']
    context = "".join(example['context']['sentences'])
    lenght = get_token_lenght(context, question)
    if lenght >= 512:
        all_pars = example['context']['title']
        gold_pars = list(set(example['supporting_facts']['title']))
        # get indices to keep from disractors
        keep_pars = gold_pars
        keep_pars_indices = [all_pars.index(keep_par) for keep_par in keep_pars]
        example['context']['title'] = [example['context']['title'][idx] for idx in keep_pars_indices]
        example['context']['sentences'] = ["".join(example['context']['sentences'][idx]) for idx in keep_pars_indices] 
    return example

In [ ]:
train_dataset = train_dataset.map(remove_distractors)
validation_dataset = validation_dataset.map(remove_distractors)

Loading cached processed dataset at /root/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/2079f58c1c29624e56735ac2c00efe8808ce52a014e1297552384edae85e3e6d/cache-d7aac0e427adf49a.arrow


Token indices sequence length is longer than the specified maximum sequence length for this model (600 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# remove yes/no questions and oversized questions
indices_train = []
indices_val = []
questions_train = train_dataset[:]['question']
contexts_train = train_dataset[:]['context']
questions_val = validation_dataset[:]['question']
contexts_val = validation_dataset[:]['context']
answers_train = train_dataset[:]['answer']
answers_val = validation_dataset[:]['answer']

for i in range(len(answers_train)):
    if answers_train[i] not in ['yes','no']:
        question = questions_train[i]
        context = "".join(contexts_train[i]['sentences'])
        lenght = get_token_lenght(context, question)
        if lenght <= 500:
            indices_train.append(i)
for i in range(len(answers_val)):
    if answers_val[i] not in ['yes','no']:
        question = questions_val[i]
        context = "".join(contexts_val[i]['sentences'])
        lenght = get_token_lenght(context, question)
        if lenght <= 500:
            indices_val.append(i)


train_dataset = train_dataset.select(indices_train)
validation_dataset = validation_dataset.select(indices_val)

In [ ]:
train_dataset

Dataset({
    features: ['answer', 'context', 'id', 'level', 'question', 'supporting_facts', 'type'],
    num_rows: 84754
})

In [ ]:
validation_dataset

Dataset({
    features: ['answer', 'context', 'id', 'level', 'question', 'supporting_facts', 'type'],
    num_rows: 6935
})

# QA model on Hotpot

## hotpot

In [ ]:
from transformers import DistilBertTokenizerFast
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

tokenizer = AutoTokenizer.from_pretrained("csarron/mobilebert-uncased-squad-v1")

model = AutoModelForQuestionAnswering.from_pretrained("csarron/mobilebert-uncased-squad-v1")

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

MobileBertForQuestionAnswering(
  (mobilebert): MobileBertModel(
    (embeddings): MobileBertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (embedding_transformation): Linear(in_features=384, out_features=512, bias=True)
      (LayerNorm): NoNorm()
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): MobileBertEncoder(
      (layer): ModuleList(
        (0): MobileBertLayer(
          (attention): MobileBertAttention(
            (self): MobileBertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=512, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MobileBertSelfOutput(
              (dense): Linear(in_features=128, o

In [ ]:
def read_hotpot(hotpot_dataset):

    contexts = []
    questions = []
    answers = []
    all_contexts = hotpot_dataset['context'][:]
    all_questions = hotpot_dataset['question'][:]
    all_answers = hotpot_dataset['answer'][:]
    invalid_quests = 0
    for i in range(len(all_contexts)):

        answer = {}
        context = "".join(all_contexts[i]['sentences'])
        answer['text'] = all_answers[i]
        answer_start = context.find(answer['text'])
        if answer_start == -1:
            invalid_quests += 1
        else:
            answer['answer_start'] = answer_start
            answers.append(answer)
            contexts.append(context)
            questions.append(all_questions[i])
    print(f'invalid percentage {invalid_quests/len(hotpot_dataset)*100:.2f}')
    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_hotpot(train_dataset[:70000])
trainval_contexts, trainval_questions, trainval_answers = read_hotpot(train_dataset[70000:])
val_contexts, val_questions, val_answers = read_hotpot(validation_dataset[:])

invalid percentage 0.00
invalid percentage 0.00
invalid percentage 0.00


## Prep data and train

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
    
add_end_idx(train_answers, train_contexts)
add_end_idx(trainval_answers, trainval_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
train_encodings = tokenizer(train_questions, train_contexts,  truncation=True, padding=True, return_tensors="pt")
trainval_encodings = tokenizer(trainval_questions, trainval_contexts,  truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(val_questions, val_contexts, truncation=True, padding=True)
val_ans_encodings = tokenizer([val_answers[i]['text'] for i in range(len(val_answers))])
train_ans_encodings = tokenizer([train_answers[i]['text'] for i in range(len(train_answers))])
trainval_ans_encodings = tokenizer([trainval_answers[i]['text'] for i in range(len(trainval_answers))])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
def add_token_positions(encodings, answers):
    def find_answer_in_context(context,answer):
        for j in range(len(context)):
            if context[j:j+len(answer)] == answer:
                return j
        return None     
    start_positions = []
    end_positions = []
    for i in range(len(answers[:])):
        ans = answers[i].tokens[1:-2]
        cntxt = encodings[i].tokens
        start_pos =  find_answer_in_context(cntxt, ans)
        start_positions.append(start_pos)

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = 0
            end_positions.append(0)
        else:
            end_positions.append(start_pos + len(ans))

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_ans_encodings)
add_token_positions(val_encodings, val_ans_encodings)
add_token_positions(trainval_encodings, trainval_ans_encodings)

In [ ]:
import torch

class HotpotDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)
BATCH_SIZE = 8
train_dataset = HotpotDataset(train_encodings)
trainval_dataset = HotpotDataset(trainval_encodings)
val_dataset = HotpotDataset(val_encodings)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
trainval_loader = DataLoader(trainval_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)



def evaluate_truth_pred(truths, preds):
    '''
    truths, preds: matched arrays of ground truth answers and predictions
    '''
    f1 = exact_match = total = 0
    for i in range(len(truths)):
        total += 1
        ground_truths = [truths[i]]
        prediction = preds[i]
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / (total+ 0.00000000001)
    f1 = 100.0 * f1 / (total+ 0.00000000001)

    return {'exact_match': exact_match, 'f1': f1}


def get_answer( model, tokenizer, context, question):
    # 1. TOKENIZE THE INPUT
    # note: if you don't include return_tensors='pt' you'll get a list of lists which is easier for
    # exploration but you cannot feed that into a model.
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    inputs = inputs.to(device)
    # 2. OBTAIN MODEL SCORES
    # the AutoModelForQuestionAnswering class includes a span predictor on top of the model.
    # the model returns answer start and end scores for each word in the text
    answer_start_scores, answer_end_scores = model(**inputs, return_dict=False)
    answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
    # 3. GET THE ANSWER SPAN
    # once we have the most likely start and end tokens, we grab all the tokens between them
    # and convert tokens back to words!
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer
# https://huggingface.co/transformers/migration.html
def model_evaluation(model, tokenizer, questions, contexts, answers, to_print = False):
    preds = []
    my_list = list(range(len(questions)))
    #with tqdm(total=len(my_list)) as pbar:
    for ex in range(len(questions)):
        answer = get_answer(model, tokenizer, contexts[ex],questions[ex])
        preds.append(answer)
        if ex % 100 == 0 and to_print:
            print("context " +contexts[ex] )
            print("quest " +questions[ex] )
            print("truth " +answers[ex]['text'] )
            print("pred " + answer)
            #pbar.update(1)
    truths = [answers[i]['text'] for i in range(len(answers))]
    scores = evaluate_truth_pred(truths, preds)
    print(scores)
    return scores




In [ ]:
from transformers import get_linear_schedule_with_warmup
n_epochs = 30
model.train()
optimizer = AdamW(model.parameters(), lr=3e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, 1000, len(val_questions)*n_epochs/BATCH_SIZE)

from tqdm import tqdm
my_list = list(range(len(train_questions)*n_epochs))
with tqdm(total=len(my_list)) as pbar:
    for epoch in range(n_epochs):
        print("####################")
        print("New epoch")
        print("####################")
        model.train()
        for batch in val_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            answer_start_scores = outputs['start_logits'][0]
            answer_end_scores = outputs['end_logits'][0]
            answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
            loss = outputs[0]
            loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(BATCH_SIZE)
        if epoch % 4 == 0:
            model.eval()
            model_evaluation(model, tokenizer, val_questions, val_contexts, val_answers)

model.eval()
model_evaluation(model, tokenizer, val_questions, val_contexts, val_answers)
#model.eval()

  0%|          | 0/2100000 [00:00<?, ?it/s]

####################
New epoch
####################


  0%|          | 6936/2100000 [04:50<22:02:59, 26.37it/s]

{'exact_match': 47.570295602018675, 'f1': 59.0529262066622}
####################
New epoch
####################


  1%|          | 13872/2100000 [13:17<22:05:07, 26.24it/s]

...

 10%|▉         | 201144/2100000 [2:41:30<20:00:12, 26.37it/s]

{'exact_match': 68.7238644556596, 'f1': 74.00583397366152}
####################
New epoch
####################


 10%|▉         | 208080/2100000 [2:50:00<25:45:41, 20.40it/s]


{'exact_match': 68.7238644556596, 'f1': 73.99819130923794}


{'exact_match': 68.7238644556596, 'f1': 73.99819130923794}

## Evaluate

In [ ]:
model.eval()
model_evaluation(model, tokenizer, val_questions, val_contexts, val_answers)

100%|██████████| 6935/6935 [05:12<00:00, 22.17it/s]


{'exact_match': 42.33597692862293, 'f1': 56.085130500668406}


{'exact_match': 42.33597692862293, 'f1': 56.085130500668406}

In [ ]:
model.eval()
model_evaluation(model, tokenizer, train_questions[:1000], train_contexts[:1000], train_answers[:1000])

100%|██████████| 1000/1000 [01:27<00:00, 11.42it/s]

{'exact_match': 56.2, 'f1': 65.28558802220684}


{'exact_match': 56.2, 'f1': 65.28558802220684}

before any training: {'exact_match': 42.33597692862293, 'f1': 56.085130500668406}


# Teaching setup

## Kernel

In [ ]:
tokenizer_sent = AutoTokenizer.from_pretrained("sentence-transformers/stsb-distilbert-base")

model_sent = AutoModel.from_pretrained("sentence-transformers/stsb-distilbert-base").to(device)

In [ ]:
def mean_pooling(model_output, attention_mask):
    #Mean Pooling - Take attention mask into account for correct averaging

    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_sentence_embedding(model, tokenizer, input):

    #Tokenize sentences
    encoded_input = tokenizer(input, return_tensors='pt').to(device)

    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True, output_hidden_states  = True)

    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output['hidden_states'], encoded_input['attention_mask'])
    return sentence_embeddings[0]
    
def get_example_embedding(model, tokenizer, question, context):
    #Tokenize sentences
    encoded_input = tokenizer(question, context, return_tensors='pt').to(device)

    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True, output_hidden_states  = True)

    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output['hidden_states'], encoded_input['attention_mask'])
    return sentence_embeddings[0]

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel
def kernel_similarity(x, y, model = None, tokenizer = None, already_embedded = True):
    if already_embedded:
        kernel_dist = rbf_kernel(x.reshape(1, -1),y.reshape(1, -1))
        return kernel_dist[0][0]
    else:
        '''
        x,y: tuple of question and context
        '''
        xq = get_sentence_embedding(model, tokenizer, x[0]).cpu().numpy()
        xc = get_sentence_embedding(model, tokenizer, x[1]).cpu().numpy()
        yq = get_sentence_embedding(model, tokenizer, y[0]).cpu().numpy()
        yc = get_sentence_embedding(model, tokenizer, y[1]).cpu().numpy()
        kernel_dist = rbf_kernel(xq.reshape(1, -1),yq.reshape(1, -1)) + rbf_kernel(xc.reshape(1, -1),yc.reshape(1, -1))
        kernel_dist /= 2
        return kernel_dist[0][0]

In [ ]:
question = " Are both Upper Crust Pizzeria and Eatza Pizza restaurants?"
context = "Upper Crust Pizzeria is a Boston, Massachusetts based chain of pizzeria restaurants. Eatza Pizza was a buffet-style restaurant chain founded in Arizona in 1997."
question2 = "Are Alek Keshishian and Kirk Kerkorian both Armenian-American?"
context2 = "Alek Keshishian (Armenian: Ալեք Գևորգի Քեշիշյան , born 30 July 1964 in Beirut, Lebanon) is an Armenian-American film and commercial director, writer, producer and music video director. Kerkor Kirk Kerkorian (June 6, 1917 – June 15, 2015) was an Armenian-American businessman, investor, and philanthropist."

In [ ]:
question = " Are both Upper Crust Pizzeria and Eatza Pizza restaurants?"
context = "Upper Crust Pizzeria is a Boston, Massachusetts based chain of pizzeria restaurants. Eatza Pizza was a buffet-style restaurant chain founded in Arizona in 1997."
question2 = "Where did the character Xavin changed into a black female just for the sake first appeared"
context2 ="Xavin had first appeared to the Runaways, taking on the form of a black male, but changed into a black female just for the sake of Karolina Dean, a lesbian whom they were to marry. Karolina Dean ( ), also briefly known as Lucy in the Sky, or L.S.D., is a fictional character, a superheroine appearing in American comic books published by Marvel Comics. The character first appeared in the series Runaways."

In [ ]:
embed1 = get_example_embedding(model, tokenizer, question, context)
embed2 = get_example_embedding(model, tokenizer, question2, context2)

In [ ]:
kernel_similarity(embed1.cpu().numpy(),embed2.cpu().numpy())

0.9706599

In [ ]:
kernel_similarity((question,context), (question2,context2), model_sent, tokenizer_sent, False)

0.95894337

## Set cover

K-mediods and k-center

In [ ]:
!pip install scikit-learn-extra

In [ ]:
# get embeddings for validation set 
my_list = list(range(len(val_questions[:])))
val_embeddings = []

with tqdm(total=len(my_list)) as pbar:
    for i in range(len(val_questions[:])):
        embed = get_example_embedding(model, tokenizer, val_questions[i], val_contexts[i]).cpu().numpy()
        val_embeddings.append(embed)
        pbar.update(1)

val_embeddings = np.asarray(val_embeddings)
from sklearn_extra.cluster import KMedoids
kmedoids = KMedoids(n_clusters=100, init='k-medoids++', max_iter=10000).fit(val_embeddings)
print(kmedoids.inertia_)
example_indices = kmedoids.medoid_indices_


100%|██████████| 6935/6935 [04:12<00:00, 27.47it/s]


11324.806


In [ ]:
from sklearn_extra.cluster import KMedoids
kmedoids = KMedoids(n_clusters=100, init='k-medoids++', max_iter=10000).fit(val_embeddings)
print(kmedoids.inertia_)
example_indices = kmedoids.medoid_indices_


11382.537


## Human model Complex


In [ ]:
# downloads
spacy.load('en')
parser = English()
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# see https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
#import pyLDAvis.gensim and https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def detect_question(token):
    # assume token is string
    tag = nltk.pos_tag([token])
    if tag[0][1] in ["WP", "WRB", "WP$"]:
        return True
    else:
        return False
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if (token not in en_stop) or  (detect_question(token))]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

def learn_lda_model():
    file_name = "../squad/train-v2.0.json"
    paragraphs = []
    with open(file_name) as f:
        dataset = json.load(f)['data']
    for article in dataset:
        for paragraph in article['paragraphs']:
            paragraphs.append(paragraph['context'])

    # prep data
    paragraphs_lda = []
    for i in range(len(paragraphs)):
        paragraphs_lda.append(prepare_text_for_lda(paragraphs[i]))

    dictionary = corpora.Dictionary(paragraphs_lda)
    corpus = [dictionary.doc2bow(text) for text in paragraphs_lda]
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('dictionary.gensim')
    NUM_TOPICS = 10
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save('lda_model_10_squad.gensim')
    topics = ldamodel.print_topics(num_words=10)
    for topic in topics:
        print(topic)

class LDA_model:
    def __init__(self):
        self.n_topics = 10
        self.model =  models.LdaModel.load('lda_model_10_squad.gensim')
        self.dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
        self.corpus = pickle.load(open('corpus.pkl', 'rb'))
    def predict(self, paragraph):
        paragraph = prepare_text_for_lda(paragraph)
        new_doc_bow = self.dictionary.doc2bow(paragraph)
        result = self.model.get_document_topics(new_doc_bow)
        vector = [0] * self.n_topics
        for top in result:
            vector[top[0]] = top[1]
        return vector
    def show_topics(self):
        topics = self.model.print_topics(num_words=10)
        for topic in topics:
            print(topic)
#learn_lda_model()


In [ ]:

#import pyLDAvis.gensim

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def detect_question(token):
    # assume token is string
    tag = nltk.pos_tag([token])
    if tag[0][1] in ["WP", "WRB", "WP$"]:
        return True
    else:
        return False



def question_classification(question):
    question_words = ["what", "when", "where", "which", "who", "whom", "whose", "why", "how"]
    question_vec = [0] * len(question_words)
    tokens = tokenize(question)
    tokens = [get_lemma(token) for token in tokens]
    all_zero = True
    for token in tokens:
        for i in range(len(question_words)):
            if token == question_words[i]:
                question_vec[i] == 1
                all_zero = False
    if all_zero:
        print(f' question: {question}')
    return question_vec, all_zero

def question_test():
    file_name = "../squad/dev-v2.0.json"
    paragraphs = []
    total = 0
    zeros = 0
    with open(file_name) as f:
        dataset = json.load(f)['data']
    for article in dataset:
        print(article["title"])
        for paragraph in article['paragraphs']:
            paragraphs.append(paragraph['context'])
            for qa in paragraph['qas']:
                total +=1
                a, b = question_classification(qa['question'])
                zeros += int(b)
    print(zeros/total*1.0)


## Human model Naiive

In [ ]:
class HumanPredictor_simple:
    def __init__(self, accuracy):
        '''
        accuracy and ai_accuracy: floats in [0,1]
        '''
        self.accuracy = accuracy
    def predict(self, contexts, questions, answers):
        '''
        expects array of strings for inputs
        returns list of string answer
        '''
        preds = []
        for i in range(len(questions)):
            coin = random.random() # random number between [0,1]
            if coin <= self.accuracy:
                preds.append(answers[i]['text'])
            else:
                # generate random answer
                sents = nltk.sent_tokenize(contexts[i])
                rand_sent = random.randint(0,len(sents)-1)
                preds.append(sents[rand_sent])
        return preds
    def prior_rejector(self, contexts, questions, answers):
        '''
        this rejector is agnostic to input, random deferall 1-self.accuracy times
        '''
        preds = []
        for i in range(len(questions)):
            coin = random.random() # random number between [0,1]
            if coin >= self.accuracy:
                preds.append(1)
            else:
                preds.append(0)
        return preds

## Our Approach

In [ ]:
class HumanLearner:
    def __init__(self, kernel):
        '''
        kernel: function that takes two inputs and returns a similarity
        prior rejector: returns rejector
        '''
        self.teaching_set = []
        self.kernel = kernel
    def predict(self, xs, prior_rejector_preds, to_print = False):
        '''
        xs: expected array of inputs
        '''
        preds = []
        idx = 0
        used_posterior = 0 
        if to_print:
            print("-- Human making reject predictions --")
            with tqdm(total=len(xs)) as pbar:
                for x in xs:
                    ball_at_x = []
                    similarities = rbf_kernel(x.reshape(1,-1), np.asarray([self.teaching_set[kk][0] for kk in range(len(self.teaching_set))]))[0]
                    for i in range(len(self.teaching_set)):
                        similarity = similarities[i]
                        if similarity >=  self.teaching_set[i][2]:
                            ball_at_x.append(self.teaching_set[i])
                    if len(ball_at_x) == 0: 
                        # use prior rejector
                        preds.append(prior_rejector_preds[idx])
                    else:
                        used_posterior += 1
                        ball_similarities = rbf_kernel(x.reshape(1,-1), np.asarray([ball_at_x[kk][0] for kk in range(len(ball_at_x))]))[0]
                        normalization = np.sum([ball_similarities[i] for i in range(len(ball_at_x))])
                        score_one = np.sum([ball_similarities[i]*ball_at_x[i][1] for i in range(len(ball_at_x))])
                        pred = score_one / normalization
                        if pred >= 0.5:
                            preds.append(1)
                        else:
                            preds.append(0)
                    idx += 1
                    pbar.update(1)
        else:
            for x in xs:
                ball_at_x = []
                similarities = rbf_kernel(x.reshape(1,-1), np.asarray([self.teaching_set[kk][0] for kk in range(len(self.teaching_set))]))[0]
                for i in range(len(self.teaching_set)):
                    similarity = similarities[i]
                    if similarity >=  self.teaching_set[i][2]:
                        ball_at_x.append(self.teaching_set[i])
                if len(ball_at_x) == 0: 
                    # use prior rejector
                    preds.append(prior_rejector_preds[idx])
                else:
                    used_posterior += 1
                    ball_similarities = rbf_kernel(x.reshape(1,-1), np.asarray([ball_at_x[kk][0] for kk in range(len(ball_at_x))]))[0]
                    normalization = np.sum([ball_similarities[i] for i in range(len(ball_at_x))])
                    score_one = np.sum([ball_similarities[i]*ball_at_x[i][1] for i in range(len(ball_at_x))])
                    pred = score_one / normalization
                    if pred >= 0.5:
                        preds.append(1)
                    else:
                        preds.append(0)
                idx += 1
        if to_print:
            print(f'Used posterior {used_posterior/len(xs)*100:.2f}')
        return preds

    def add_to_teaching(self, teaching_example):
        '''
        teaching_example: (x, label, gamma)
        '''
        self.teaching_set.append(teaching_example)

    def remove_last_teaching_item(self):
        self.teaching_set = self.teaching_set[:-1]

Exception ignored in: <function tqdm.__del__ at 0x7fd391e02710>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1062, in __del__
    self.close()
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1237, in close
    if self.disable:
AttributeError: 'tqdm' object has no attribute 'disable'


In [ ]:
def compute_predictions_humanai(hum_preds, hum_rejector, ai_preds, data_x):
    '''
    hum_preds: array of human predictions
    ai_preds: array of AI predictions
    hum_rejector: HumanLearner
    data_x: array of inputs

    Returns array of final predictions and deferalls
    '''
    predictions = []
    with torch.no_grad():
        reject_decisions = hum_rejector(data_x)
        for i in range(len(data_x)):
            if reject_decisions[i] == 1:
                # defer
                predictions.append(ai_preds[i])
            else:
                predictions.append(hum_preds[i])
    return predictions, reject_decisions

def get_metrics(preds, truths):
    # custom for each use case
    return evaluate_truth_pred(truths, preds)

def compute_metrics(human_preds, ai_preds, reject_decisions, truths, to_print = False):
    coverage = 1 - np.sum(reject_decisions)/len(reject_decisions)
    humanai_preds = []
    human_preds_sys = []
    truths_human = []
    ai_preds_sys = []
    truths_ai = []
    for i in range(len(reject_decisions)):
        if reject_decisions[i] == 1:
            humanai_preds.append(ai_preds[i])
            ai_preds_sys.append(ai_preds[i])
            truths_ai.append(truths[i])
        else:
            humanai_preds.append(human_preds[i])
            human_preds_sys.append(human_preds[i])
            truths_human.append(truths[i])
    humanai_metrics = get_metrics(humanai_preds, truths)
    human_metrics = get_metrics(human_preds_sys, truths_human)
    ai_metrics = get_metrics(ai_preds_sys, truths_ai)
    if to_print:
        print(f'Coverage is {coverage*100:.2f}')
        print(f' metrics of system are: {humanai_metrics}')
        print(f' metrics of human are: {human_metrics}')
        print(f' metrics of AI are: {ai_metrics}')
    return coverage, humanai_metrics, human_metrics, ai_metrics

In [ ]:
# Preliminary steps:
# Step 1.1: get predictions of human and predictions of AI on teaching set
# Step 1.2: get prior rejector decisions of Human on teaching set
# Step 2: Compute optimal deferal decision for each point in teaching set  
# Step 3: Compute optimal gamma for each point, can start with a constant guess

# Algorithms:
# Initialize HumanLearner
# Baseline 1:
# step 1: get k-mediods points
# step 2: add to HumanLearner
# step 3: evaluate on validation set

In [ ]:
# Define teaching and validation
split_teach_val = 0.5
teaching_contexts = trainval_contexts[:math.floor(len(trainval_contexts)*split_teach_val)]
teaching_questions = trainval_questions[:math.floor(len(trainval_contexts)*split_teach_val)]
teaching_answers = trainval_answers[:math.floor(len(trainval_contexts)*split_teach_val)]
validation_contexts = trainval_contexts[math.floor(len(trainval_contexts)*split_teach_val):]
validation_questions = trainval_questions[math.floor(len(trainval_contexts)*split_teach_val):]
validation_answers = trainval_answers[math.floor(len(trainval_contexts)*split_teach_val):]



In [ ]:
# Get AI predictions
ai_teaching_preds = []
ai_validation_preds = []
print("--Getting AI predictions on teach and validation sets \n--")
with tqdm(total=len(teaching_contexts)) as pbar:
    for ex in range(len(teaching_contexts)):
        answer = get_answer(model, tokenizer, teaching_contexts[ex], teaching_questions[ex])
        ai_teaching_preds.append(answer)
        pbar.update(1)
with tqdm(total=len(validation_contexts)) as pbar:
    for ex in range(len(validation_contexts)):
        answer = get_answer(model, tokenizer, validation_contexts[ex], validation_questions[ex])
        ai_validation_preds.append(answer)
        pbar.update(1)

  0%|          | 2/7377 [00:00<08:09, 15.06it/s]

--Getting AI predictions on teach and validation sets 
--


100%|██████████| 7377/7377 [05:51<00:00, 20.97it/s]


In [ ]:
# Get Human Predictions



# Get AI predictions
ai_teaching_preds = []
ai_validation_preds = []
ai_val_preds = []
ai_train_preds = []

with open('C:/Users/Hussein/Documents/Research/Human learn to defer/hotpotqa/outputs_val_1dis/predictions_ans.json', 'r') as handle:
    preds_val = json.load(handle)
    
#with open('preds_train_data.p', 'rb') as handle:
#    preds_train = pickle.load(handle)

for key, value in preds_val.items():
    ai_val_preds.append(value)

#for key, value in preds_train.items():
#    ai_train_preds.append(value)

ai_teaching_preds = ai_val_preds[:START_TEST]
ai_validation_preds = ai_val_preds[START_TEST:]





human_predictor = HumanPredictor_simple(0.8)
hum_teaching_preds = []
hum_validation_preds = []
priorhum_teaching_preds = []
priorhum_validation_preds = []


--Getting Human predictions on teach and validation sets --

--Getting prior rejector predictions on teach and validation sets


In [ ]:
# Optimal deferall decisions
opt_defer_teaching = []
opt_defer_validation = []
for ex in range(len(teaching_contexts)):
    f1_hum = metric_max_over_ground_truths(f1_score, teaching_answers[ex]['text'], [hum_teaching_preds[ex]])
    f1_ai = metric_max_over_ground_truths(f1_score, teaching_answers[ex]['text'], [ai_teaching_preds[ex]])

    if f1_ai > f1_hum:
        opt_defer_teaching.append(1)
    else:
        opt_defer_teaching.append(0)

for ex in range(len(validation_contexts)):
    f1_hum = metric_max_over_ground_truths(f1_score, validation_answers[ex]['text'], [hum_validation_preds[ex]])
    f1_ai = metric_max_over_ground_truths(f1_score, validation_answers[ex]['text'], [ai_validation_preds[ex]])
    if f1_ai > f1_hum:
        opt_defer_validation.append(1)
    else:
        opt_defer_validation.append(0)

In [ ]:
compute_metrics(hum_validation_preds, ai_validation_preds, priorhum_validation_preds, [validation_answers[i]['text'] for i in range(len(validation_answers))])

Coverage is 79.83
 metrics of system are: {'exact_match': 75.58628168632225, 'f1': 78.23708365565689}
 metrics of human are: {'exact_match': 80.38716250636767, 'f1': 81.33845872431229}
 metrics of AI are: {'exact_match': 56.58602150537596, 'f1': 65.96289159966645}


(0.7982919886132575,
 {'exact_match': 75.58628168632225, 'f1': 78.23708365565689},
 {'exact_match': 80.38716250636767, 'f1': 81.33845872431229},
 {'exact_match': 56.58602150537596, 'f1': 65.96289159966645})

In [ ]:
compute_metrics(hum_validation_preds, ai_validation_preds, opt_defer_validation, [validation_answers[i]['text'] for i in range(len(validation_answers))])

Coverage is 36.94
 metrics of system are: {'exact_match': 91.73105598481756, 'f1': 93.86405510427629}
 metrics of human are: {'exact_match': 95.55963302752258, 'f1': 96.14810443891169}
 metrics of AI are: {'exact_match': 89.48839208942371, 'f1': 92.52612852713017}


(0.3693913514978989,
 {'exact_match': 91.73105598481756, 'f1': 93.86405510427629},
 {'exact_match': 95.55963302752258, 'f1': 96.14810443891169},
 {'exact_match': 89.48839208942371, 'f1': 92.52612852713017})

In [ ]:
# get embeddings for teaching set 
teaching_embeddings = []
with tqdm(total=len(teaching_questions)) as pbar:
    for i in range(len(teaching_questions[:])):
        embed = get_example_embedding(model, tokenizer, teaching_questions[i], teaching_contexts[i]).cpu().numpy()
        teaching_embeddings.append(embed)
        pbar.update(1)
teaching_embeddings = np.asarray(teaching_embeddings)

validation_embeddings = []
with tqdm(total=len(validation_questions)) as pbar:
    for i in range(len(validation_questions[:])):
        embed = get_example_embedding(model, tokenizer, validation_questions[i], validation_contexts[i]).cpu().numpy()
        validation_embeddings.append(embed)
        pbar.update(1)
validation_embeddings = np.asarray(validation_embeddings)


100%|██████████| 7377/7377 [04:42<00:00, 26.16it/s]


In [ ]:
# get optimal gammas
optimal_gammas = []
with tqdm(total=len(teaching_embeddings)) as pbar:
    for i in range(len(teaching_embeddings)):
        # get all similarities
        opt_defer_ex = opt_defer_teaching[i]
        similarities_embeds = rbf_kernel(teaching_embeddings[i].reshape(1,-1), np.asarray(teaching_embeddings))[0]
        sorted_sim = sorted([(similarities_embeds[k], opt_defer_teaching[k]) for k in range(len(teaching_embeddings))], key=lambda tup: tup[0])
        indicess = list(range(1, len(opt_defer_teaching)))
        indicess.reverse()
        for k in indicess:
            if sorted_sim[k][1] == opt_defer_ex and sorted_sim[k- 1][1] != opt_defer_ex:
                optimal_gammas.append(sorted_sim[k][0])
                break
        pbar.update(1)

100%|██████████| 7377/7377 [03:28<00:00, 35.35it/s]


In [ ]:
# Baseline get medoids
from sklearn_extra.cluster import KMedoids
kmedoids = KMedoids(n_clusters=10, init='k-medoids++', max_iter=10000).fit(teaching_embeddings)
print(kmedoids.inertia_)
teaching_indices = kmedoids.medoid_indices_

13291.047


In [ ]:
[teaching_contexts[i] for i in teaching_indices]

["Stabbing Westward is an American industrial rock band. Christopher Hall and Walter Flakus formed the band in 1986 in Macomb, Illinois, but did not release their first studio album until eight years later. The band released three more studio albums before announcing a dissolution on February 9, 2002. Stabbing Westward reunited in 2016 to celebrate their 30th anniversary together, and will play more shows in 2017.State Radio is a Boston-based rock trio comprising singer and primary songwriter Chad Stokes Urmston (also a member of Dispatch), bassist Chuck Fay, and, formerly, drummer Michael Najarian. The band's songs focus on social and political issues and have been musically described as a combination of reggae, punk and rock.",
 'Hugh Ferriss (1889 – 1962) was an American delineator (one who creates drawings and sketches of buildings) and architect. After his death a colleague said he \'influenced my generation of architects\' more than any other man. Ferriss also influenced popular 

In [ ]:
human_learner = HumanLearner(kernel_similarity)


In [ ]:
for teach_ex_idx in teaching_indices:
    ex_embed = teaching_embeddings[teach_ex_idx]
    ex_label = opt_defer_teaching[teach_ex_idx]
    gamma = optimal_gammas[teach_ex_idx] # random choice
    human_learner.add_to_teaching([ex_embed, ex_label, gamma])

In [ ]:
preds_teaching = human_learner.predict(teaching_embeddings, priorhum_teaching_preds)

  0%|          | 35/7377 [00:00<00:21, 346.81it/s]

-- Human making reject predictions --


100%|██████████| 7377/7377 [00:19<00:00, 384.68it/s]

Used posterior 30.61


In [ ]:
preds_val = human_learner.predict(validation_embeddings, priorhum_validation_preds)

  1%|          | 37/7377 [00:00<00:19, 368.77it/s]

-- Human making reject predictions --


100%|██████████| 7377/7377 [00:19<00:00, 382.74it/s]

Used posterior 26.56


In [ ]:
compute_metrics(hum_validation_preds, ai_validation_preds, preds_val, [validation_answers[i]['text'] for i in range(len(validation_answers))])

Coverage is 84.86
 metrics of system are: {'exact_match': 76.98251321675467, 'f1': 79.17845779927067}
 metrics of human are: {'exact_match': 80.2396166134184, 'f1': 81.20980314949209}
 metrics of AI are: {'exact_match': 58.728737690241196, 'f1': 67.79419469059745}


(0.8485834350006778,
 {'exact_match': 76.98251321675467, 'f1': 79.17845779927067},
 {'exact_match': 80.2396166134184, 'f1': 81.20980314949209},
 {'exact_match': 58.728737690241196, 'f1': 67.79419469059745})

In [ ]:
compute_metrics(hum_validation_preds, ai_validation_preds, priorhum_validation_preds, [validation_answers[i]['text'] for i in range(len(validation_answers))])

Coverage is 79.83
 metrics of system are: {'exact_match': 75.58628168632225, 'f1': 78.23708365565689}
 metrics of human are: {'exact_match': 80.38716250636767, 'f1': 81.33845872431229}
 metrics of AI are: {'exact_match': 56.58602150537596, 'f1': 65.96289159966645}


(0.7982919886132575,
 {'exact_match': 75.58628168632225, 'f1': 78.23708365565689},
 {'exact_match': 80.38716250636767, 'f1': 81.33845872431229},
 {'exact_match': 56.58602150537596, 'f1': 65.96289159966645})

In [ ]:
compute_metrics(hum_teaching_preds, ai_teaching_preds, preds_teaching, [teaching_answers[i]['text'] for i in range(len(teaching_answers))])

Coverage is 84.87
 metrics of system are: {'exact_match': 78.98874881388087, 'f1': 81.23010892614373}
 metrics of human are: {'exact_match': 82.35106213064992, 'f1': 83.35042708942146}
 metrics of AI are: {'exact_match': 60.1254480286733, 'f1': 69.33466804775396}


(0.8487189914599431,
 {'exact_match': 78.98874881388087, 'f1': 81.23010892614373},
 {'exact_match': 82.35106213064992, 'f1': 83.35042708942146},
 {'exact_match': 60.1254480286733, 'f1': 69.33466804775396})

In [ ]:
compute_metrics(hum_teaching_preds, ai_teaching_preds, priorhum_teaching_preds, [teaching_answers[i]['text'] for i in range(len(teaching_answers))])

Coverage is 79.91
 metrics of system are: {'exact_match': 75.59983733224878, 'f1': 78.37410342845895}
 metrics of human are: {'exact_match': 80.71246819338408, 'f1': 81.78910994403012}
 metrics of AI are: {'exact_match': 55.263157894736466, 'f1': 64.79012002137856}


(0.7991053273688491,
 {'exact_match': 75.59983733224878, 'f1': 78.37410342845895},
 {'exact_match': 80.71246819338408, 'f1': 81.78910994403012},
 {'exact_match': 55.263157894736466, 'f1': 64.79012002137856})

## Our approach

In [ ]:
data_sizes  = []
indices_used = []
points_chosen = []


In [ ]:
MAX_SIZE = 2000
human_learner = HumanLearner(kernel_similarity)
errors = []
for itt in range(MAX_SIZE):
    print(f'New size {itt}')
    best_index = -1
    best_value = 0

    valid_indices = list(set(list(range(len(teaching_answers)))) - set(indices_used))
    subset_size = min(10, len(valid_indices))
    random_teach_subset = random.sample(valid_indices, subset_size) # used to take gradient steps
    random_validation_subset = random.sample(valid_indices, subset_size)
    print("")
    # for each point, add and see effect then remove
    counter_ = 0
    for j in random_teach_subset:
        counter_ += 1
        ex_embed = teaching_embeddings[j]
        ex_label = opt_defer_teaching[j]
        gamma = optimal_gammas[j] # random choice
        human_learner.add_to_teaching([ex_embed, ex_label, gamma])
        preds_teach = human_learner.predict([teaching_embeddings[i] for i in random_validation_subset], [priorhum_teaching_preds[i] for i in random_validation_subset])
        _, metrics, __, ___ = compute_metrics([hum_teaching_preds[kk] for kk in random_validation_subset], [ai_teaching_preds[kk] for kk in random_validation_subset], preds_teach, [teaching_answers[kk]['text'] for kk in random_validation_subset])
        #if counter_ % 100 == 0:
        #    print(metrics)
        acc = metrics["f1"]
        if acc >= best_value:
            best_value = acc
            best_index = j
        human_learner.remove_last_teaching_item()

    indices_used.append(best_index) # add found element to set used
    ex_embed = teaching_embeddings[best_index]
    ex_label = opt_defer_teaching[best_index]
    gamma = optimal_gammas[best_index] # random choice
    human_learner.add_to_teaching([ex_embed, ex_label, gamma])
    if itt % 50 == 0:
        print("####### Actual eval " +str(itt)+ " ###########")
        preds_teach = human_learner.predict(validation_embeddings, priorhum_validation_preds)
        _, metrics, __, ___ = compute_metrics(hum_validation_preds, ai_validation_preds, preds_teach, [validation_answers[i]['text'] for i in range(len(validation_answers))], True)
        errors.append(metrics)   
        print("##############################")